In [1]:
import yaml
from run_experiment import choose_dataset_class,choose_task_classes

import torch
import numpy as np
from collections import Counter

In [22]:
# 0. read the embeddings and labesl
yaml_args= yaml.load(open('../SA-config/a-sample-pos-get_cor.yaml'))

dataset_class = choose_dataset_class(yaml_args)

task_class, reporter_class, loss_class = choose_task_classes(yaml_args)

task = task_class(yaml_args)

expt_dataset = dataset_class(yaml_args, task)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 1


[aligning embeddings]:   0%|          | 6/12543 [00:00<03:58, 52.61it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [01:48<00:00, 116.01it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   1%|          | 20/2077 [00:00<00:10, 198.41it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 357/12543 [00:00<00:03, 3568.15it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7508.47it/s]


In [24]:
# 1. prepare the label matrix.
# train_data = expt_dataset.train_dataset
train_data = expt_dataset.dev_dataset

labels=[]
for obser in train_data.observations:
    labels.append(task.labels(obser))

all_labels = torch.cat(labels, 0).numpy().astype(int)

all_labels.shape
all_labels_mat= np.zeros((all_labels.size, all_labels.max()+1))
all_labels_mat[np.arange(all_labels.size),all_labels] = 1
all_labels_mat.shape

(25150, 50)

In [25]:
# 2. prepare the embedding matrix.

embeddings=[]
for obser in train_data.observations:
    embeddings.append(obser.embeddings)

all_embeddings = torch.cat(embeddings, 0).numpy()

all_embeddings.shape

(25150, 768)

In [26]:
# 3. compute the corrcoef matrix. 
# num=204609
# corr_mat_whole = np.corrcoef(all_labels_mat[0:num].T, all_embeddings[0:num].T)
corr_mat_whole = np.corrcoef(all_labels_mat.T, all_embeddings.T)
corr_mat=np.absolute(corr_mat_whole[0:50,50:])
corr_mat.shape

(50, 768)

In [27]:
# 4. output the corr_mat data. dump as a pkl.


In [28]:
# 4. output the corr_sum_dim
# we compute the corr score of each dim for all the 50 pos labels.
dim_sum=[]
for i in range(768):
    dim_sum.append(np.sum(corr_mat[:,i]))

# corr_sum_dim=np.argsort(dim_sum)[::-1]
# with open('../SA-dim-files/average_corr_dim.tsv','w') as fout:
#     for dim in corr_sum_dim:
#         fout.write('{} '.format(dim))

In [29]:
# 5. output the weighted_average_corr_dim
# we compute the weighted average corr socre of each dim for all the pos labels

# 5.1 get the distribution/weights 
all_labels
result = Counter(all_labels)
weights = [result[i]/len(all_labels) for i in range(50)]



# 5.2 get the weighted score
dim_weighted_average=[]
for i in range(768):
    dim_weighted_average.append(np.average(corr_mat[:,i],weights=weights))
    
# # 5.3 output the dims
# dim_weighted_average_dim=np.argsort(dim_weighted_average)[::-1]
# with open('../SA-dim-files/weighted_average_corr_dim.tsv','w') as fout:
#     for dim in dim_weighted_average_dim:
#         fout.write('{} '.format(dim))

In [34]:
np.argsort(weights)

array([39, 49, 44, 42, 45, 38, 47, 31, 30, 41, 48, 32, 28, 46, 23, 19, 40,
       37, 34, 36, 35,  1, 29,  2, 33, 27, 43, 11, 17, 25, 12, 16, 20, 14,
       18,  5, 21, 24, 26,  4,  6, 13, 22, 15, 10,  3,  0,  7,  9,  8])

In [37]:
weights[8],weights[9],

(0.13284294234592445, 0.09355864811133201)

In [15]:
# 6. output the max_corr
max_val=[]
for i in range(768):
    max_val.append(np.max(corr_mat[:,i]))

dim_max_val=np.argsort(max_val)[::-1]
with open('../SA-dim-files/max_corr_dim.tsv','w') as fout:
    for dim in dim_weighted_average_dim:
        fout.write('{} '.format(dim))

In [16]:
np.max(dim_sum),np.max(dim_weighted_average),np.max(corr_mat),np.max(max_val)

(4.1708928549625375,
 0.19993299987706492,
 0.4437054850588246,
 0.4437054850588246)

In [17]:

def make_corr_file(layer=1):

    # 0. read the embeddings and labesl
    yaml_args= yaml.load(open('../SA-config/a-sample-pos-get_cor.yaml'))

    yaml_args['model']['model_layer']=layer-1

    dataset_class = choose_dataset_class(yaml_args)

    task_class, reporter_class, loss_class = choose_task_classes(yaml_args)

    task = task_class(yaml_args)

    expt_dataset = dataset_class(yaml_args, task)

    # 1. prepare the label matrix.
    train_data = expt_dataset.train_dataset
    labels=[]
    for obser in train_data.observations:
        labels.append(task.labels(obser))

    all_labels = torch.cat(labels, 0).numpy().astype(int)

    all_labels.shape
    all_labels_mat= np.zeros((all_labels.size, all_labels.max()+1))
    all_labels_mat[np.arange(all_labels.size),all_labels] = 1
    all_labels_mat.shape

    # 2. prepare the embedding matrix.

    embeddings=[]
    for obser in train_data.observations:
        embeddings.append(obser.embeddings)

    all_embeddings = torch.cat(embeddings, 0).numpy()

    all_embeddings.shape

    # 3. compute the corrcoef matrix. 
    # num=204609
    # corr_mat_whole = np.corrcoef(all_labels_mat[0:num].T, all_embeddings[0:num].T)
    corr_mat_whole = np.corrcoef(all_labels_mat.T, all_embeddings.T)
    corr_mat=np.absolute(corr_mat_whole[0:50,50:])
    corr_mat.shape

    # 4. output the corr_mat data. dump as a pkl.


    # 4. output the corr_sum_dim
    # we compute the corr score of each dim for all the 50 pos labels.
    dim_sum=[]
    for i in range(768):
        dim_sum.append(np.sum(corr_mat[:,i]))

    corr_sum_dim=np.argsort(dim_sum)[::-1]
    with open('../SA-dim-files/average_corr_dim_layer_{}.tsv'.format(layer),'w') as fout:
        for dim in corr_sum_dim:
            fout.write('{} '.format(dim))

    # 5. output the weighted_average_corr_dim
    # we compute the weighted average corr socre of each dim for all the pos labels

    # 5.1 get the distribution/weights 
    all_labels
    result = Counter(all_labels)
    weights = [result[i]/len(all_labels) for i in range(50)]



    # 5.2 get the weighted score
    dim_weighted_average=[]
    for i in range(768):
        dim_weighted_average.append(np.average(corr_mat[:,i],weights=weights))

    # 5.3 output the dims
    dim_weighted_average_dim=np.argsort(dim_weighted_average)[::-1]
    with open('../SA-dim-files/weighted_average_corr_dim_layer_{}.tsv'.format(layer),'w') as fout:
        for dim in dim_weighted_average_dim:
            fout.write('{} '.format(dim))

    # 6. output the max_corr
    max_val=[]
    for i in range(768):
        max_val.append(np.max(corr_mat[:,i]))

    dim_max_val=np.argsort(max_val)[::-1]
    with open('../SA-dim-files/max_corr_dim_layer_{}.tsv'.format(layer),'w') as fout:
        for dim in dim_weighted_average_dim:
            fout.write('{} '.format(dim))

In [18]:
make_corr_file(layer=1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 0


[aligning embeddings]:   0%|          | 34/12543 [00:00<00:36, 338.81it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:29<00:00, 421.56it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   2%|▏         | 42/2077 [00:00<00:04, 417.28it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 350/12543 [00:00<00:03, 3494.16it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7565.72it/s]


In [19]:
make_corr_file(layer=3)
make_corr_file(layer=6)
make_corr_file(layer=9)
make_corr_file(layer=12)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 2


[aligning embeddings]:   0%|          | 30/12543 [00:00<00:42, 292.45it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:29<00:00, 421.01it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   0%|          | 8/2077 [00:00<00:33, 62.13it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 336/12543 [00:00<00:03, 3356.18it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7230.32it/s]


In [20]:
for layer in [1,2,4,5,7,8,10,11]:
    make_corr_file(layer=layer)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 0


[aligning embeddings]:   0%|          | 34/12543 [00:00<00:37, 337.34it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:30<00:00, 417.41it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   0%|          | 7/2077 [00:00<00:30, 66.78it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 353/12543 [00:00<00:03, 3529.08it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7256.78it/s]


## study the distribution of the embeddings on different layers

In [2]:

def get_corr_map(layer=1):

    # 0. read the embeddings and labesl
    yaml_args= yaml.load(open('../SA-config/a-sample-pos-get_cor.yaml'))

    yaml_args['model']['model_layer']=layer-1

    dataset_class = choose_dataset_class(yaml_args)

    task_class, reporter_class, loss_class = choose_task_classes(yaml_args)

    task = task_class(yaml_args)

    expt_dataset = dataset_class(yaml_args, task)

    # 1. prepare the label matrix.
    train_data = expt_dataset.train_dataset
    labels=[]
    for obser in train_data.observations:
        labels.append(task.labels(obser))

    all_labels = torch.cat(labels, 0).numpy().astype(int)

    all_labels.shape
    all_labels_mat= np.zeros((all_labels.size, all_labels.max()+1))
    all_labels_mat[np.arange(all_labels.size),all_labels] = 1
    all_labels_mat.shape

    # 2. prepare the embedding matrix.

    embeddings=[]
    for obser in train_data.observations:
        embeddings.append(obser.embeddings)

    all_embeddings = torch.cat(embeddings, 0).numpy()

    all_embeddings.shape

    # 3. compute the corrcoef matrix. 
    # num=204609
    # corr_mat_whole = np.corrcoef(all_labels_mat[0:num].T, all_embeddings[0:num].T)
    corr_mat_whole = np.corrcoef(all_labels_mat.T, all_embeddings.T)
    return corr_mat_whole,all_embeddings,all_labels_mat

In [3]:
corr_map_layer_dict=dict.fromkeys([i+1 for i in range(12)])

In [4]:
for layer in [1,3,6,9,12]:

    corr_mat_whole,all_embeddings,all_labels_mat=get_corr_map(layer=layer)
    corr_map_layer_dict[layer]={
        'corr_mat_whole':corr_mat_whole,
        'all_embeddings':all_embeddings,
        'all_labels_mat':all_labels_mat
    }

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 0


The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
[aligning embeddings]:   0%|          | 35/12543 [00:00<00:36, 346.89it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:29<00:00, 423.09it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   2%|▏         | 42/2077 [00:00<00:04, 415.71it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 332/12543 [00:00<00:03, 3316.66it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7096.98it/s]


In [50]:
for layer in [2,4,5,7,8,10,11]:

    corr_mat_whole,all_embeddings,all_labels_mat=get_corr_map(layer=layer)
    corr_map_layer_dict[layer]={
        'corr_mat_whole':corr_mat_whole,
        'all_embeddings':all_embeddings,
        'all_labels_mat':all_labels_mat
    }

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 1


[aligning embeddings]:   0%|          | 37/12543 [00:00<00:34, 365.28it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:29<00:00, 429.82it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   2%|▏         | 44/2077 [00:00<00:04, 438.29it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 365/12543 [00:00<00:03, 3648.66it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7300.48it/s]


In [18]:
for layer in [1,3,6,9,12]:
#     emb_mat = np.absolute(corr_map_layer_dict[layer]['corr_mat_whole'][50:,50:]-np.eye(768, dtype=int))
    emb_mat = corr_map_layer_dict[layer]['corr_mat_whole'][50:,50:]-np.eye(768, dtype=int)

    print('layer: ',layer,np.max(emb_mat),np.min(emb_mat),np.mean(np.absolute(emb_mat)),np.median(np.absolute(emb_mat)))
    

layer:  1 0.9344386443610329 -0.44777127433163433 0.05445517548586958 0.045042605675376095
layer:  3 0.5388708180055392 -0.3891944787099866 0.04346215384211578 0.03648069088313568
layer:  6 0.40894972665598045 -0.46745444691679733 0.040155630284109615 0.0335834003641113
layer:  9 0.29406288335072756 -0.5437658693230691 0.037730351200416595 0.03145815035875132
layer:  12 0.8915102125630212 -0.9125223359281933 0.062358668342203924 0.050669063341169114


In [19]:
def get_top_dim_from_file(file):
    with open(file,'r') as fin:
        lines = fin.readlines()
        line=lines[0].strip().split()
        dims=[int(line[i]) for i in range(len(line))]
    return dims
        

In [64]:

for layer in [1,2,3,4,5,6,7,8,9,10,11,12]:
    k=4
#     emb_mat = np.absolute(corr_map_layer_dict[layer]['corr_mat_whole'][50:,50:]-np.eye(768, dtype=int))
    emb_mat = np.array(corr_map_layer_dict[layer]['corr_mat_whole'][50:,50:]-np.eye(768, dtype=int))
    dim_file = '../SA-dim-files/average_corr_dim_layer_{}.tsv'.format(layer)
    print('layer\t{}\tmax-corr\t{}\tmean-corr\t{} '.format(layer,np.max(np.absolute(emb_mat)),np.mean(np.absolute(emb_mat)),))
#     print('layer: ',layer,np.max(emb_mat),np.min(emb_mat),np.max(np.absolute(emb_mat)),np.mean(np.absolute(emb_mat)),np.median(np.absolute(emb_mat)))
    dims=get_top_dim_from_file(file=dim_file)
    top_emb_mat = emb_mat[dims[0:k]]
    top_emb_mat = top_emb_mat[:,dims[0:k]]
#     print('layer: ',layer,np.max(top_emb_mat),np.min(top_emb_mat),np.max(np.absolute(top_emb_mat)),np.mean(np.absolute(top_emb_mat)),np.median(np.absolute(top_emb_mat)))
#     print(top_emb_mat)
    

layer	1	max-corr	0.9344386443610329	mean-corr	0.05445517548586958 
layer	2	max-corr	0.6790318898745872	mean-corr	0.045984754656778135 
layer	3	max-corr	0.5388708180055392	mean-corr	0.04346215384211578 
layer	4	max-corr	0.45349813123061394	mean-corr	0.04306083598380115 
layer	5	max-corr	0.4081479121011531	mean-corr	0.0412659191877765 
layer	6	max-corr	0.46745444691679733	mean-corr	0.040155630284109615 
layer	7	max-corr	0.5212013234748754	mean-corr	0.03904785366661185 
layer	8	max-corr	0.5658149898225272	mean-corr	0.03822041388538651 
layer	9	max-corr	0.5437658693230691	mean-corr	0.037730351200416595 
layer	10	max-corr	0.7716978112891458	mean-corr	0.03926388344173654 
layer	11	max-corr	0.7371881250346611	mean-corr	0.041128799093268414 
layer	12	max-corr	0.9125223359281933	mean-corr	0.062358668342203924 


In [65]:

for layer in [1,2,3,4,5,6,7,8,9,10,11,12]:
    k=5
#     emb_mat = np.absolute(corr_map_layer_dict[layer]['corr_mat_whole'][50:,50:]-np.eye(768, dtype=int))
    emb_mat = np.array(corr_map_layer_dict[layer]['corr_mat_whole'][50:,50:]-np.eye(768, dtype=int))
    dim_file = '../SA-dim-files/average_corr_dim_layer_{}.tsv'.format(layer)
#     print('layer: ',layer,np.max(emb_mat),np.min(emb_mat),np.max(np.absolute(emb_mat)),np.mean(np.absolute(emb_mat)),np.median(np.absolute(emb_mat)))
    dims=get_top_dim_from_file(file=dim_file)
    top_emb_mat = emb_mat[dims[0:k]]
    top_emb_mat = top_emb_mat[:,dims[0:k]]
    print('layer\t{}\tmax-corr\t{}\tmean-corr\t{} '.format(layer,np.max(np.absolute(top_emb_mat)),np.mean(np.absolute(top_emb_mat)),))
#     print('layer: ',layer,np.max(top_emb_mat),np.min(top_emb_mat),np.max(np.absolute(top_emb_mat)),np.mean(np.absolute(top_emb_mat)),np.median(np.absolute(top_emb_mat)))
#     print(top_emb_mat)
    

layer	1	max-corr	0.9344386443610329	mean-corr	0.33377593890588725 
layer	2	max-corr	0.6790318898745872	mean-corr	0.2774187195087825 
layer	3	max-corr	0.5388708180055392	mean-corr	0.13675694176581563 
layer	4	max-corr	0.45349813123061394	mean-corr	0.11756843120368758 
layer	5	max-corr	0.3974267527739602	mean-corr	0.14666693370442338 
layer	6	max-corr	0.40894972665598045	mean-corr	0.1272581291682241 
layer	7	max-corr	0.3823351255817417	mean-corr	0.12069361046873045 
layer	8	max-corr	0.31555230120604344	mean-corr	0.11244593569842101 
layer	9	max-corr	0.2464554089840436	mean-corr	0.1098964963130136 
layer	10	max-corr	0.2551217223482187	mean-corr	0.12510328069650808 
layer	11	max-corr	0.21970662681363654	mean-corr	0.08209500617508024 
layer	12	max-corr	0.18384732208722365	mean-corr	0.06815413873166584 
